# Testing the Algorithm 
To test if the algorithm works properly and in alignment with the instructions from the Project Guidelines, **3 tests** will be run. These include:  
- **2 functional tests**: does the algorithm output correct solutions if adequate inputs are provided? This especially pertains to the amount of transactions (are they really minimized) and the path that it outputs (are the correct amounts transferred to and from the correct people). Functional tests will be verified through comparison with the actual Splitwise-app. 
- **1 performance test**: does the algorithm scale well? I.e. can it solve transactions with 10, 25 or 50 different involved people in under a second?

## Exception Tests


## Functional Tests

## Performance Test
Does the algorithm solve transactions with more than 15 people in less than 1 second? 

In [5]:
import random, time
from splitwise import OptimalSplit

def generate_transactions(num_people, num_transactions_per_person=10, max_amount=100):
    """
    Generate a list of transactions of the form [lender, borrower, amount].

    Args:
      - num_people: how many unique people (named "P0", "P1", …).
      - num_transactions_per_person: average number of transactions per person.
      - max_amount: upper bound for a single transaction’s amount.
    """
    people = [f"P{i}" for i in range(num_people)]
    transactions = []
    for _ in range(num_people * num_transactions_per_person):
        lender, borrower = random.sample(people, 2)
        amount = round(random.uniform(1, max_amount), 2)
        transactions.append([lender, borrower, amount])
    return transactions

transactions_1 = generate_transactions(5, 5, 100)
transactions_2 = generate_transactions(15, 5, 100)
transactions_3 = generate_transactions(50, 5, 100)

print(transactions_2)


[['P4', 'P10', 58.72], ['P5', 'P14', 72.75], ['P5', 'P7', 72.36], ['P9', 'P10', 87.81], ['P6', 'P11', 59.02], ['P5', 'P6', 27.06], ['P1', 'P4', 93.2], ['P2', 'P1', 23.11], ['P0', 'P8', 33.09], ['P8', 'P1', 5.42], ['P5', 'P3', 58.1], ['P5', 'P0', 13.93], ['P5', 'P6', 91.01], ['P7', 'P14', 44.56], ['P2', 'P8', 49.11], ['P3', 'P13', 76.63], ['P2', 'P6', 51.67], ['P8', 'P2', 75.75], ['P0', 'P7', 16.43], ['P1', 'P0', 32.94], ['P6', 'P3', 32.63], ['P4', 'P7', 59.14], ['P14', 'P7', 60.46], ['P6', 'P5', 99.45], ['P2', 'P5', 75.5], ['P11', 'P4', 64.78], ['P3', 'P0', 51.97], ['P5', 'P10', 65.06], ['P8', 'P10', 1.72], ['P7', 'P11', 96.54], ['P4', 'P7', 6.56], ['P5', 'P12', 42.31], ['P5', 'P1', 17.08], ['P2', 'P12', 45.48], ['P3', 'P1', 58.19], ['P13', 'P1', 73.06], ['P12', 'P9', 50.64], ['P8', 'P10', 18.44], ['P2', 'P3', 2.04], ['P9', 'P7', 98.34], ['P8', 'P6', 29.61], ['P9', 'P10', 97.58], ['P7', 'P5', 41.77], ['P7', 'P1', 11.8], ['P7', 'P9', 14.12], ['P0', 'P7', 25.67], ['P12', 'P5', 17.65], ['

In [6]:
solver = OptimalSplit()
for n in (5, 15, 50):
    txns = generate_transactions(n)
    start = time.perf_counter()
    solver.minTransfers(txns)
    elapsed = time.perf_counter() - start
    print(f"{n:2d} people → {elapsed:.4f}s")
    assert elapsed < 1.0, f"Took too long for {n} people"

defaultdict(<class 'int'>, {'P2': -581.0699999999999, 'P1': 34.16999999999998, 'P0': -112.86999999999999, 'P4': 743.65, 'P3': -83.88000000000001})
{'P2': -581.0699999999999, 'P1': 34.16999999999998, 'P0': -112.86999999999999, 'P4': 743.65, 'P3': -83.88000000000001}
 5 people → 0.0001s
defaultdict(<class 'int'>, {'P12': -41.44999999999999, 'P7': -33.80000000000007, 'P9': 333.32, 'P10': -490.54999999999995, 'P6': 77.34, 'P5': 99.43000000000006, 'P13': -418.99, 'P4': -28.569999999999908, 'P2': -41.680000000000035, 'P0': 258.44, 'P11': 198.74, 'P3': 48.49000000000003, 'P8': -194.57999999999998, 'P14': 386.37, 'P1': -152.51})
{'P12': -41.44999999999999, 'P7': -33.80000000000007, 'P9': 333.32, 'P10': -490.54999999999995, 'P6': 77.34, 'P5': 99.43000000000006, 'P13': -418.99, 'P4': -28.569999999999908, 'P2': -41.680000000000035, 'P0': 258.44, 'P11': 198.74, 'P3': 48.49000000000003, 'P8': -194.57999999999998, 'P14': 386.37, 'P1': -152.51}
15 people → 0.0001s
defaultdict(<class 'int'>, {'P18': -

In [7]:
transactions = [['P0', 'P8', 57.84], ['P5', 'P3', 18.98], ['P4', 'P14', 82.19], ['P9', 'P2', 37.96], ['P4', 'P11', 90.33], ['P11', 'P9', 74.92], ['P5', 'P6', 79.66], ['P11', 'P14', 2.82], ['P7', 'P8', 35.58], ['P11', 'P10', 61.73], ['P7', 'P1', 32.3], ['P3', 'P10', 92.2], ['P9', 'P4', 20.2], ['P5', 'P8', 11.16], ['P11', 'P10', 78.52], ['P11', 'P6', 67.42], ['P12', 'P7', 60.01], ['P0', 'P6', 48.12], ['P5', 'P4', 35.8], ['P10', 'P5', 12.62], ['P2', 'P0', 28.55], ['P11', 'P1', 16.15], ['P6', 'P4', 10.49], ['P13', 'P4', 21.64], ['P9', 'P13', 38.32], ['P4', 'P8', 19.9], ['P7', 'P2', 91.32], ['P8', 'P6', 77.25], ['P2', 'P11', 72.95], ['P7', 'P10', 29.73], ['P13', 'P4', 6.42], ['P7', 'P10', 42.13], ['P12', 'P4', 53.18], ['P11', 'P9', 2.27], ['P6', 'P11', 92.43], ['P1', 'P4', 20.77], ['P11', 'P12', 73.13], ['P5', 'P7', 77.82], ['P5', 'P11', 63.83], ['P13', 'P10', 73.35], ['P3', 'P10', 63.91], ['P11', 'P13', 90.25], ['P13', 'P8', 39.55], ['P5', 'P3', 59.41], ['P0', 'P12', 5.77], ['P9', 'P13', 2.21], ['P9', 'P6', 48.88], ['P12', 'P5', 31.7], ['P9', 'P0', 39.94], ['P5', 'P12', 22.19], ['P2', 'P6', 83.47], ['P1', 'P13', 37.59], ['P0', 'P5', 32.21], ['P11', 'P8', 31.23], ['P3', 'P13', 18.99], ['P13', 'P0', 3.99], ['P5', 'P9', 93.63], ['P4', 'P6', 36.84], ['P0', 'P11', 87.0], ['P8', 'P1', 10.07], ['P13', 'P7', 6.87], ['P14', 'P13', 81.91], ['P5', 'P2', 71.36], ['P13', 'P10', 26.41], ['P12', 'P14', 46.7], ['P8', 'P6', 87.2], ['P5', 'P7', 76.77], ['P10', 'P13', 69.5], ['P6', 'P1', 41.12], ['P10', 'P0', 40.94], ['P4', 'P3', 50.6], ['P6', 'P14', 34.04], ['P0', 'P12', 94.45], ['P8', 'P11', 80.79], ['P4', 'P8', 56.86]]

solver.minTransfers(transactions)

defaultdict(<class 'int'>, {'P0': 211.97000000000003, 'P8': 3.190000000000012, 'P5': 534.08, 'P3': 46.10999999999999, 'P4': 168.21999999999997, 'P14': -83.84, 'P9': 16.689999999999998, 'P2': -15.669999999999987, 'P11': 11.109999999999943, 'P6': -350.76, 'P7': 9.58999999999999, 'P10': -344.91999999999996, 'P1': -41.27999999999999, 'P12': -3.9499999999999886, 'P13': -160.54000000000002})
{'P0': 211.97000000000003, 'P8': 3.190000000000012, 'P5': 534.08, 'P3': 46.10999999999999, 'P4': 168.21999999999997, 'P14': -83.84, 'P9': 16.689999999999998, 'P2': -15.669999999999987, 'P11': 11.109999999999943, 'P6': -350.76, 'P7': 9.58999999999999, 'P10': -344.91999999999996, 'P1': -41.27999999999999, 'P12': -3.9499999999999886, 'P13': -160.54000000000002}


[['P5', 'P6', 350.76],
 ['P0', 'P10', 211.97000000000003],
 ['P5', 'P13', 160.54000000000002],
 ['P4', 'P10', 132.94999999999993],
 ['P3', 'P14', 46.10999999999999],
 ['P4', 'P1', 35.27000000000004],
 ['P5', 'P14', 22.78000000000003],
 ['P9', 'P2', 15.669999999999987],
 ['P11', 'P14', 11.109999999999943],
 ['P7', 'P1', 6.009999999999948],
 ['P7', 'P12', 3.580000000000041],
 ['P8', 'P14', 3.190000000000012],
 ['P9', 'P14', 0.650000000000027],
 ['P9', 'P12', 0.3699999999999477]]